# Animal Attacks in North America

In [213]:
#import packages
from lxml import html
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [214]:
#function to get rows in a table
def get_table_rows(table):
    l = []
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    return l

In [215]:
#function to get rows in table (works for multiple tables)
def get_rows_table(tables):
    tb=[]
    for barbie in tables:
        tables=barbie.findNext('table')
        tb.append(tables)
    l = []

    for table in tb:
        table_rows = table.find_all('tr')
        row = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
    return l

In [216]:
#install packages
#!pip install geocoder
#!conda install -c conda-forge folium=0.5.0 --yes

In [217]:
#import packages to draw on map
import geocoder
import folium
from geopy.geocoders import Nominatim

In [232]:
# create map of North America using latitude and longitude values
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="na_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_north_america = folium.Map(location=[latitude, longitude], zoom_start=3)
map_north_america_pure = map_north_america

In [219]:
def add_coordinates(table):
    lat_lng_coords = None
    latlong=[]
    for n in range(0,len(table)):
        g = geocoder.arcgis(table['Location'].values[n])
        lat_lng_coords = g.latlng
        latlong.append(lat_lng_coords)
    laa = np.array(latlong)
    latitude=laa[:,0].tolist()
    longitude=laa[:,1].tolist()
    lat=pd.DataFrame(latitude,columns=['Latitude']) 
    long=pd.DataFrame(longitude,columns=['Longitude']) 
    lulo=pd.concat([lat,long], axis=1)
    data=pd.concat([table,lulo],axis=1)
    return data

In [220]:
def add_to_map(table,colour,fill_colour,map_of):
    for lat, lng, Loc in zip(table['Latitude'], table['Longitude'], table['Location']):
        label = '{}'.format(Loc)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng], radius=3, popup=label, color=colour, fill=True, fill_color=fill_colour, fill_opacity=0.7, parse_html=False).add_to(map_of)
    return map_of

##  Bear Attacks in North America

Scrape and Explore Dataset

In [221]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_fatal_bear_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [222]:
dflist=[]
for bear_type in ("Polar","Black","Brown"):
    bears=soup.findAll(title="Edit section: "+bear_type+" bear")
    bear=get_rows_table(bears)
    df=pd.DataFrame(bear, columns=["Person", "Date", "Type", "Location", "Details"])
    df['Bear']=bear_type
    dflist.append(df)
all_attacks=pd.concat(dflist).replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [223]:
data=add_coordinates(all_attacks)
data=data.loc[data['Type']=='Wild'].reset_index(drop=True)
pb=data.loc[data['Bear']=='Polar']
blb=data.loc[data['Bear']=='Black']
brb=data.loc[data['Bear']=='Brown']

In [224]:
map_bear_attacks=add_to_map(brb,'brown','#3186cc',add_to_map(blb,'black','#3186cc',add_to_map(pb,'blue','#3186cc',map_north_america)))
map_bear_attacks

# Wolf Attacks

In [225]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_wolf_attacks_in_North_America")
soup = BeautifulSoup(r.content, "html.parser")

In [226]:
fatals=soup.find('table')
df=pd.DataFrame(get_rows_table(fatals), columns=["Victim(s)","Age","Sex","Date","Type of attack","Location","Details","Source(s)"])
ftl_wolf_attacks = df.replace(r'\n',' ', regex=True).dropna().reset_index(drop=True)

In [227]:
wvs=add_coordinates(ftl_wolf_attacks)

In [233]:
#add_to_map(wvs,'gray','gray',map_north_america)
add_to_map(wvs,'gray','gray',map_north_america_pure)